# This Notebook contains all solutions from the Exercise 3

## Nr. 1

- Pick a list within the Wikipedia like the list of sovereign states. Choose some other list on your own, based on your personal interests. The only requirement is that there are other Wikipedia articles linked within the list.
## Nr.2

- Get all the names and URLs to the corresponding items in the list and export them into a CSV file that has two columns (name and URL).

In [1]:
import requests
from bs4 import BeautifulSoup
import csv
url = "https://en.wikipedia.org/wiki/List_of_countries_with_alcohol_prohibition"
content = requests.get(url).text
soup = BeautifulSoup(content)
def titles(soup):
    present_list = soup.find_all("ul")[1]
    titles = []
    for x in present_list.find_all("a"):
        titles.append(x.get("title"))
    titles = list(set(titles))
    remove = [None, "Indonesia", "COVID-19 pandemic in South Africa", "India", "South Yemen", "Union Territory", "Sharjah (emirate)"]
    for things in remove:
        titles.remove(things)
    titles_clean = sorted(titles)
    return titles_clean
def href(titles):
    url = "https://wikipedia.org/wiki/"
    href = []
    links = []
    for country in titles:
        href.append(url)
    for i in range(len(href)):
        links.append(list(href[i] + titles[x] for x in range(len(titles))))
    links = links.pop(0)
    links_clean = []
    for entries in links:
        re = entries.replace(" ", "_")
        links_clean.append(re)
    return links_clean
def writer(title, link):
    column_names = ["Names", "URL"]
    with open("List_of_countries_with_alcohol_prohibition.csv", "w", newline = "") as csvfile:
        writer = csv.writer(csvfile, delimiter=",")
        writer.writerow(column_names)
        writer.writerows(zip(title, link))
titles(soup)
href(titles(soup))
writer(titles(soup), href(titles(soup)))

## Nr. 3

- For every Wikipedia article in the CSV list choose a few attributes from the infobox on the right that you would like to extract (e.g., population, name of the head of state, whatever...). Extract this information for every entry in your list. Store this information in an appropriate data structure.

In [2]:
from bs4 import BeautifulSoup
import csv
import pandas as pd
import requests
import re
import lxml
url = "https://en.wikipedia.org/wiki/List_of_countries_with_alcohol_prohibition"
content = requests.get(url).text
soup = BeautifulSoup(content)
def titles(soup):
    present_list = soup.find_all("ul")[1]
    titles = []
    for x in present_list.find_all("a"):
        titles.append(x.get("title"))
    titles = list(set(titles))
    remove = [None, "Indonesia", "COVID-19 pandemic in South Africa", "India", "South Yemen", "Union Territory", "Sharjah (emirate)"]
    for things in remove:
        titles.remove(things)
    titles_clean = sorted(titles)
    return titles_clean
def href(titles):
    url = "https://wikipedia.org/wiki/"
    href = []
    links = []
    for countrie in titles:
        href.append(url)
    for i in range(len(href)):
        links.append(list(href[i] + titles[x] for x in range(len(titles))))
    links = links.pop(0)
    links_clean = []
    for entries in links:
        repl = entries.replace(" ", "_")
        links_clean.append(repl)
    return links_clean
titles(soup)
href(titles(soup))
def capital(link):
    content = requests.get(link).text
    soup = BeautifulSoup(content, "lxml")
    infobox = soup.find("table", attrs={"class": "infobox geography vcard"}).tbody
    rows = infobox.find_all("tr")
    capitals = []
    for x in rows:
        if re.search("Capital", str(x)) != None:
            if x.find("a").get_text() == "Capital":
                capitals.append(x.find_all(["a"])[1].get_text())
            else:
                capitals.append(x.find("a").get_text())
    capitals_clean = []
    for entries in capitals:
        repl = entries.replace(" ", "_")
        capitals_clean.append(repl)
    for x in capitals_clean:
        return x
def timezone(link):
    content = requests.get(link).text
    soup = BeautifulSoup(content, "lxml")
    infobox = soup.find("table", attrs={"class": "infobox geography vcard"}).tbody
    rows = infobox.find_all("tr")
    timezone = []
    for x in rows:
        if re.search("Time zone", str(x)) != None:
            if x.find(["span", "a"]).get_text() == "Time zone":
                timezone.append(x.find_all(["span", "a"])[1].get_text())
            else:
                timezone.append(x.find(["span", "a"]).get_text())
    timezones_clean = []
    for entries in timezone:
        repl = entries.replace(" ", "_")
        timezones_clean.append(repl)
    for x in timezones_clean:
        return x
def writer(title, link, capitals, timezones):
    column_names = ["Names", "URL", "Capitals", "Timezones"]
    with open("List_of_countries_with_alcohol_prohibition.csv", "w", newline = "") as csvfile:
        writer = csv.writer(csvfile, delimiter=",")
        writer.writerow(column_names)
        writer.writerows(zip(title, link, capitals, timezones))
capitals = []
timezones = []
df = pd.read_csv("List_of_countries_with_alcohol_prohibition.csv", delimiter = ",")
for link in df["URL"]:
    capitals.append(capital(link))
    timezones.append(timezone(link))
writer(titles(soup), href(titles(soup)), capitals, timezones)

## Nr. 4

- Save your scraped information into a JSON file. Try to export clean data.

In [3]:
import csv
import json
CSV_PATH = './List_of_countries_with_alcohol_prohibition.csv'
JSON_PATH = './List_of_countries_with_alcohol_prohibition.json'
csv_file = csv.DictReader(open(CSV_PATH, 'r'))
json_list = []
for row in csv_file:
    json_list.append(row)
open(JSON_PATH, 'w').write(json.dumps(json_list, indent=4))

3810